In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
file = "data/exoplanet.eu_catalog.csv"

In [3]:
data = pd.read_csv(file, skiprows=0, usecols=['radius', 'orbital_period', 'ra', 'dec', 'mag_v', 
                                              'star_distance', 'star_radius', 'semi_major_axis',
                                              'eccentricity', 'tzero_tr'])

In [4]:
# We want to drop rows with null planet and stellar radii
# We also need the V-band magnitudes
data = data.dropna(subset={"radius", "star_radius", "mag_v", "tzero_tr"})
# We want only stars with mag_v ~< 12
data = data[data["mag_v"] < 12]

# Now compute transit depth for each
transit_depth = (0.10049*data["radius"] / data["star_radius"])**2
# Compute transit signature in magnitudes
transit_sig = -2.5 * np.log10(1 - transit_depth)
# Drop all rows where transit signature is < 0.008
data = data[transit_sig > 0.008]

In [5]:
semi_minor_axis = data["semi_major_axis"] * np.sqrt(1 - data["eccentricity"]**2)
# This is the lower bound on circumference
circ = np.sqrt(2) * np.pi * np.sqrt(data["semi_major_axis"]**2 + semi_minor_axis**2)
# Velocity in meters per hour
veloc = circ / data["orbital_period"] * 1.496e11 / 24
# Very rough duration estimate
transit_dur = 2 * data["star_radius"] * 6.957e8 / veloc
# Keep transit duration under 3 hours
data = data[transit_dur < 3]

In [6]:
data = data[data["dec"] > -19]

In [7]:
def from_jd(jd):
    
    A = jd // 1
    L = A+68569
    N = 4*L//146097
    L = L-(146097*N+3)//4
    I = 4000*(L+1)//1461001
    L = L-1461*I//4+31
    J = 80*L//2447
    K = L-2447*J//80
    L = J//11
    J = J+2-12*L
    I = 100*(N-49)+I+L
    
    B = jd % 1
    hour = B * 24
    minute = (hour % 1) * 60
    second = (minute % 1) * 60
    
    I = map(int, I)
    J = map(int, J)
    K = map(int, K)
    hour = map(int, hour)
    minute = map(int, minute)
    second = map(int, second)
    
    return [datetime(i, j, k, h, m, s) for i, j, k, h, m, s
            in zip(I, J, K, hour, minute, second)]

def equinox(year):
    
    return datetime(year, 3, 21, 0, 0, 0)

def delta_ra(dates):
    
    dts = (dateobj - equinox(dateobj.year) for dateobj in dates)
    return np.array([dt.days for dt in dts])

dra = delta_ra(from_jd(data["tzero_tr"]))
ra = (dra * 360 / 365.25 + 180 + 75) % 360

for i, r in zip(data.index, ra):
    
    print(i, r)

data = data[ra > 285]

312 190.93429158110882
320 8.34702258726901
326 225.4312114989733
328 223.45995893223818
329 195.86242299794662
336 199.80492813141683
695 64.52772073921972
3773 205.71868583162217
3825 353.56262833675567
3826 33.97330595482549
3856 287.52566735112936
3859 33.97330595482549
3870 60.585215605749454
3892 305.26694045174537
3934 67.48459958932239
3939 61.57084188911705
3944 312.1663244353183
3946 130.56468172484597
3947 297.3819301848049
3969 171.96098562628345
3989 359.476386036961
4002 10.31827515400414
4047 247.11498973305956
4048 221.48870636550308


In [8]:
# September 5th at 19:00
after = 2458731.375
# Compute the next transit
rem = (after - data["tzero_tr"]) % data["orbital_period"]
obsday = after - rem + data["orbital_period"]

for i in obsday.index:
    
    while not (0.3 <= obsday[i] % 1 <= 0.5): obsday[i] += data["orbital_period"][i]
        
for i, date in zip(obsday.index, from_jd(obsday)):
    
    print(i, date)

3825 2019-10-23 07:40:15
3856 2019-09-10 07:12:56
3892 2019-09-06 11:07:53
3944 2019-09-04 09:26:34
3947 2019-09-14 09:30:01
3989 2019-09-27 09:02:37


In [9]:
data  

,radius,orbital_period,semi_major_axis,eccentricity,tzero_tr,ra,dec,mag_v,star_distance,star_radius
3825,1.099,3.030072,0.03930,0.000,2.453187e+06,286.037500,36.632500,11.79,157.0,0.850
3856,1.137,1.755414,0.02918,0.000,2.456406e+06,160.602542,7.435083,11.12,143.0,0.930
3892,1.281,2.243766,0.03600,0.087,2.454964e+06,218.275000,21.894722,9.75,160.0,1.306
3944,1.454,1.846837,0.03130,0.000,2.454606e+06,278.633333,35.661667,10.64,223.0,1.310
3947,1.603,1.219870,0.02558,0.000,2.454590e+06,36.712500,37.550556,8.30,116.0,1.444
3989,1.057,3.868138,0.04525,0.000,2.455749e+06,315.025000,-5.094444,9.87,50.0,0.813
